In [ ]:
import numpy as np
from tensor import Tensor
from nn import Linear
from tqdm import trange
from datasets import load_dataset

l1 = Linear(784, 128)
l2 = Linear(128, 10)

x = Tensor(np.random.uniform(-1., 1., size=(784)).astype(np.float32))
x = l1(x)
# print(x.data.shape)
x = x.relu()
# print(x.data.shape)
x = l2(x)
x = x.logsoftmax()
x = x.mean()
x.backward()

In [ ]:
x = Tensor(np.random.uniform(-1., 1., size=(784)).astype(np.float32))
l1w = Tensor(np.random.uniform(-1., 1.,size=(784, 128)).astype(np.float32))
# bias should be 1d and 0 initialized
l1b = Tensor(np.zeros((128), dtype=np.float32))

x = x.dot(l1w.data.T)
print(x.data.shape)
x = x.add(l1b)
print(x.data.shape)

x = x.mean()
x.backward()

In [8]:
l1.params['b'].grad

In [ ]:
l1.params['w']

In [4]:
# make a tensor of size 128, 128 and fill it with zero
a = Tensor(np.zeros((128, 128)))
# make a tensor of size 128 and fill it with one
b = Tensor(np.ones((128,128)))
# add them together
c = a.add(b)
c = c.mean()
c.backward()

In [5]:
b.grad

array([[6.10351562e-05, 6.10351562e-05, 6.10351562e-05, ...,
        6.10351562e-05, 6.10351562e-05, 6.10351562e-05],
       [6.10351562e-05, 6.10351562e-05, 6.10351562e-05, ...,
        6.10351562e-05, 6.10351562e-05, 6.10351562e-05],
       [6.10351562e-05, 6.10351562e-05, 6.10351562e-05, ...,
        6.10351562e-05, 6.10351562e-05, 6.10351562e-05],
       ...,
       [6.10351562e-05, 6.10351562e-05, 6.10351562e-05, ...,
        6.10351562e-05, 6.10351562e-05, 6.10351562e-05],
       [6.10351562e-05, 6.10351562e-05, 6.10351562e-05, ...,
        6.10351562e-05, 6.10351562e-05, 6.10351562e-05],
       [6.10351562e-05, 6.10351562e-05, 6.10351562e-05, ...,
        6.10351562e-05, 6.10351562e-05, 6.10351562e-05]])

In [1]:
from tensor import Tensor
import numpy as np

a = Tensor(np.array([1,2,3]))
b = Tensor(np.array([4,5,6]))
c = a.add(b)
print(c)
c.mean().backward()

Saved tensors: [array([1, 2, 3]), array([4, 5, 6])]
ctx: <tensor.Add object at 0x000001AD3FD387F0>
Tensor [5 7 9] with grad None
Saved tensors: [array([5, 7, 9])]
ctx: <tensor.Sum object at 0x000001AD3FD50940>
Saved tensors: [array([21]), array([0.33333333])]
ctx: <tensor.Mul object at 0x000001AD5F1062B0>
